<a href="https://colab.research.google.com/github/naufalhisyam/TurbidityPrediction-thesis/blob/main/model_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
base_model = tf.keras.applications.ResNet50(include_top=False, weights="imagenet",
                                            input_shape=(224, 224, 3), pooling=None)
base_model.trainable = False
inputs = keras.Input(shape=(150, 150, 3))